# transform kc, ps to graph

In [ ]:
import pandas as pd
import networkx as nx
import json
from pathlib import Path
from unidecode import unidecode
from networkx.drawing.nx_pydot import graphviz_layout

prefix = 'data/umimematikucz-system_'

kc = pd.read_csv(Path(prefix + "kc.csv", ), sep=';', header=0, index_col=0)
follow = pd.read_csv(Path(prefix + "kc_follow.csv", ), sep=';', header=0, index_col=0)
#ps = pd.read_csv(Path(prefix + "ps.csv", ), sep=';', header=0, index_col=0)

SIZE = (100, 100)
def recursively_change_shape(node, shape):
    nx.set_node_attributes(G, {node:shape}, name='shape')
    for pred in G.predecessors(node):
        recursively_change_shape(pred, shape)

shapes = ['diamond', 'triangle', 'rectangle', 'ellipse', 'octagon', 'v', 'hexagon', 'parallelogram', 'round-rectangle']
G = nx.DiGraph(size=SIZE)

G.add_node(0, label='"Root"', color='black', shape='ellipse')
G.add_nodes_from([(int(idx), {'label':f'"{unidecode(name)}"', 'color': 'black', 'shape':'ellipse'}) for idx, name in zip(kc.index, kc['name'])])
G.add_edges_from([(int(idx), int(parent)) for idx, parent in zip(kc.index, kc['parent'])], color='black')

for idx, shape in zip(G.predecessors(0), shapes):
    recursively_change_shape(idx, shape)

G.add_edges_from([(int(first), int(second)) for first, second in follow[['kc1', 'kc2']].to_numpy()], color='silver')
pos = graphviz_layout(G, prog="dot")

# application

In [ ]:
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc
import src.custom_components as cc
from dash import dcc, html, Input, Output
from dash import callback_context as ctx
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate
import src.graph as g
from src.layout import make_layout
from src.utils import dbc_upload

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

styles = json.load(open('styles.json', 'r'))

app.layout = make_layout(styles)

In [ ]:
from base64 import b64decode
import io

@app.callback(
    Output('graph_elems', 'data'),
    Input('upload_graph', 'contents'),
    # prevent_initial_call=True,  TODO uncomment
)
def upload_graph(upload):
    """
    TODO uncomment and update graph.json
    return dbc_upload(upload)
    """
    return g.graph_as_elements(G, pos)  # the cake is a lie


@app.callback(
    Output('download', 'data'),
    Input('graph_elems', 'data'),
    Input('download_btn', 'n_clicks'),
    prevent_initial_call=True,
)
def download_graph(data, _):
    if ctx.triggered_id == 'download_btn':
        return dcc.send_string(json.dumps(data), filename='updated_graph.json')
    raise PreventUpdate


@app.callback(
    Output('test', 'children'),
    Input('cytoscape', 'elements'),
)
def test_out(inp):
    return ''


def modify_by_id(elements, id, key, value):
    elems = [elem for elem in elements if elem['data']['id'] == id]
    if len(elems) != 1:
        raise RuntimeError(f'Detected {len(elems)} elements for {id=}. Expected 1.')
    elem = elems[0]
    elem['data'][key] = value
    return [elem for elem in elements if elem['data']['id'] != id] + [elem]

@app.callback(
        Output('cytoscape', 'elements'),
        Input('mode_btn', 'value'),
        Input('cytoscape', 'selectedNodeData'),
        Input('cytoscape', 'elements'),
        Input('graph_elems', 'data'),
        Input('layout_select', 'value'),
        prevent_initial_call=True,
)
def update_graph(mode, selected, elements, new_elements, layout):
    if ctx.triggered_id == 'graph_elems':
        graph = g.graph_from_elements(new_elements, size=SIZE)
        position = graphviz_layout(graph, prog=layout)
        return g.graph_as_elements(graph, position)
    
    if ctx.triggered_id == 'layout_select':
        graph = g.graph_from_elements(elements, size=SIZE)
        position = graphviz_layout(graph, prog=layout)
        return g.graph_as_elements(graph, position)

    if ctx.triggered[0]['prop_id'].split('.')[1] == 'selectedNodeData':
        graph = g.graph_from_elements(elements, size=SIZE)
        for i in range(len(elements)):
            if g.is_element_edge(elements[i]):
                continue
            elif any(elements[i]['data']['id'] == s['id'] for s in selected):
                elements[i]['data']['color'] = 'red'
            elif any(g.graph_index_from_elements(elements[i]) in graph.successors(g.graph_index_from_elements(s)) for s in selected):
                elements[i]['data']['color'] = 'orange'
            elif any(g.graph_index_from_elements(elements[i]) in graph.predecessors(g.graph_index_from_elements(s)) for s in selected):
                elements[i]['data']['color'] = 'yellow'
            else:
                elements[i]['data']['color'] = 'black'

        if mode == 1 and len(selected) >= 2:
            edge = g.graph_edge_as_elements(g.graph_index_from_elements(selected[-2]), g.graph_index_from_elements(selected[-1]), {'color':'blue'})
            old_length = len(elements)
            elements = g.filter_edge_from_elements(edge, elements)
            if len(elements) == old_length:
                elements.append(edge)
            
        return elements

    raise PreventUpdate


In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True)